# Inicialização do Modelo

In [ ]:
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from sklearn.metrics import classification_report
import json
from settings import API_KEY


# try:
#     llm  = ChatOllama(model="gemma3:27b", temperature=0, base_url="http://192.168.133.192:11435")
# except Exception as e:
#     print(f"Erro ao inicializar o Ollama. Certifique-se de que o Ollama está em execução e o modelo 'gemma3' está disponível.")
#     print(f"Detalhes do erro: {e}")
#     exit()

my_key = API_KEY
MODEL_NAME = "gpt-3.5-turbo"

try:
    llm = ChatOpenAI(api_key= my_key, 
                   model=MODEL_NAME, 
                   temperature=0,
                )
except Exception as e:
    print("Erro ao inicializar o groq")
    print(f"Detalhes do erro: {e}")
    exit()

In [2]:
llm.invoke("bom dia")

AIMessage(content='Bom dia! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Be4jGWW6wN0M27Hf49L5agxAO5z7K', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5b85133e-c912-4b84-ac30-a39e24292c08-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
def load_conll_data(filepath):
    """
    Carrega dados de um ficheiro no formato CoNLL.

    Args:
        filepath (str): O caminho para o ficheiro .txt.

    Returns:
        tuple: Uma tupla contendo uma lista de frases (listas de palavras)
               e uma lista de sequências de etiquetas.
    """

    setences, tag_sequences = [], []
    current_words, current_tags = [], []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if  line:  # Linha vazia
                    if len(line.split()) == 2:
                        word, tag = line.split()
                        current_words.append(word)
                        current_tags.append(tag)
            else:
                 if current_words:
                    setences.append(current_words)
                    tag_sequences.append(current_tags)
                    current_words, current_tags = [], []
    if current_words:
        setences.append(current_words)
        tag_sequences.append(current_tags)

    return setences, tag_sequences


palavras, tag = load_conll_data("test.txt")

# Definição dos Prompts

In [4]:
system_prompt_text = """

Você é um especialista em Reconhecimento de Entidades Nomeadas (NER) treinado para identificar e classificar entidades em textos jurídicos brasileiros.
Sua tarefa é analisar o texto fornecido e retornar cada palavra (token) acompanhada de uma etiqueta NER no formato BIO.
As etiquetas possíveis são:

- O: Para palavras que não pertencem a nenhuma entidade.
- B-Organizacao: Para a primeira palavra em uma frase de uma entidade do tipo "Organização" (ex: Ministério Público, Tribunal de Justiça).
- I-Organizacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Organização".
- B-Jurisprudencia: Para a primeira palavra em uma frase de uma entidade do tipo "Jurisprudência" (ex: Súmula 385 STJ, Recurso Especial nº 1.234.567).
- I-Jurisprudencia: Para palavras subsequentes dentro de uma frse para entidade do tipo "Jurisprudência".
- B-Pessoa: Para a primeira palavra em uma frase de uma entidade do tipo "Pessoa" (ex: João da Silva, Maria Oliveira).
- I-Pessoa: Para palavras subsequentes dentro de uma frase para entidade do tipo "Pessoa".
- B-Tempo: Para a primeira palavra em uma frase de uma entidade do tipo "Tempo" (ex: 01 de janeiro de 2023, segunda-feira, prazo de 15 dias).
- I-Tempo: Para palavras subsequentes dentro de uma frase para entidade do tipo "Tempo".
- B-Local: Para a primeira palavra em uma frase de uma entidade do tipo "Local" (ex: São Paulo, Comarca de Campinas, Brasil).
- I-Local: Para palavras subsequentes dentro de uma frase para entidade do tipo "Local".
- B-Legislacao: Para a primeira palavra em uma frase de uma entidade do tipo "Legislação" (ex: Código Civil, Lei nº 8.666/93, Art. 5º da Constituição Federal).
- I-Legislacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Legislação".

A entrada será uma lista de listas de string no seguinte formato:
[
   Frase1: ['O', 'Dia', 'Está', 'Lindo', 'Hoje'],
   Frase2: ['O', 'Relator', 'Está', 'Ciente', 'Do', 'Contrato'],
   Frase3: ['No', 'caso', 'em', 'debate', 'nestes', 'autos', ',', 'a', 'guarda']
   Frase4: ['Eu', 'sou', 'uma', 'pessoa', 'muito', 'inteligente', '-', 'disse', 'o', 'homem']
   Frase5: ['O', 'arts','.','58','e','67', ',', 'Lei', '8666/94', ',' , '186', 'e', '927', 'do', 'Código', 'Civil', ')', '.' ]
   Frase6: ['Encontra-se', 'em', 'consonância', 'com', 'o', 'fundamento', 'acolhido', 'pelo', 'STF']
   Frase7: ['Confiram-se', 'as', 'seguintes', 'decisões', ':', 'Rcl', '13941', 'MC', '/', ',' , 'Relator', 'Ministros', 'Cezar', 'Peluso', 'DJE', '31/08/2012']
]

A saída DEVE ser estritamente no formato, e apenas nesse formato e nada mais:
{{

    "Frase1": ["O", "O", "O", "O", "B-TEMPO"]
    "Frase2": ["O", "B-PESSOA", "O", "O", "O"]
    "Frase3": ["O","O","O","O","O","O","O","O","O"]
    "Frase4": ["B-PESSOA", "O", "O", "I-PESSOA", "O", "O", "O", "O", "O", "I-PESSOA"]
    "Frase5": ["O", "B-LEGISLACAO", "I-LEGISLACAO", "I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","O"]
    "Frase6": ["O", "O", "O", "O", "O", "O", "O", "O", "B-ORGANIZAÇÃO"]
    "Frase7": ["O", "O", "O", "O", "O", "B-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "O", "O", "O", "B-PESSOA", "I-PESSOA", "O", "B-TEMPO"]
}}

Cada palavra do texto original deve estar em uma nova linha, seguida por um espaço e sua respectiva etiqueta NER.
Se uma palavra for um sinal de pontuação, como vírgula, ponto final ou hífen, atribua a etiqueta "O", a menos que seja parte intrínseca de uma entidade nomeada (o que é raro).
Analise cuidadosamente o contexto para determinar as fronteiras corretas das entidades.
Preste atenção à capitalização, pois pode ser um indicador de entidade, mas não confie apenas nisso.
Entidades podem ser compostas por múltiplas palavras, então note que podemos ter vírgulas ou pontos significando coisas diferentes dependendo do contexto na frase.
Preste atenção para não gerar menos etiquetas do que palavras, ou seja, se houver 10 palavras, deve haver 10 etiquetas.

NÃO RESPONDA EM MARKDOWN, APENAS EM STRING SIMPLES.
"""

user_prompt_text = """Analise o seguinte texto e forneça a classificação NER para cada palavra no formato especificado:
TEXTO DE ENTRADA:
{texto_entrada}"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("user", user_prompt_text)
])


# Criação da Chain

In [5]:
chain = prompt_template | llm | StrOutputParser()


# Criação da lógica de tratamento das frases para LLM

In [6]:
def format_batch_for_llm(batch_sentences, start_index=1):
    """
    Formata um lote de frases para a string de entrada esperada pelo LLM.
    Args:
        batch_sentences (list): Uma lista de frases (onde cada frase é uma lista de palavras).
        start_index (int): O índice inicial para nomear as frases (Frase1, Frase2, ...).
    Returns:
        str: A string formatada para o LLM.
    """
    formatted_lines = []
    for i, sentence_words in enumerate(batch_sentences):
        # Escapar aspas simples dentro das palavras, se houver
        escaped_sentence_words = [word.replace("'", "\\'") for word in sentence_words]
        formatted_lines.append(f"    Frase{start_index + i}: {escaped_sentence_words}")
    
    return "[\n" + ",\n".join(formatted_lines) + "\n]"


# --- Função principal para processamento ---
def process_ner_in_batches(filepath, batch_size=10):
    print(f"A carregar frases e etiquetas verdadeiras do ficheiro: {filepath}")
    all_sentences, all_original_tags = load_conll_data(filepath)

    if not all_sentences:
        print("Nenhuma frase encontrada no ficheiro. A sair.")
        return

    print(f"Total de frases carregadas: {len(all_sentences)}")
    
    all_true_tags_for_report = []
    all_predicted_tags_for_report = []

    for i in range(0, len(all_sentences), batch_size):
        batch_sentences = all_sentences[i : i + batch_size]
        batch_true_tags = all_original_tags[i : i + batch_size] # Etiquetas verdadeiras para este lote
        
        # Usar i+1 como start_index para ter chaves globais no JSON (Frase1, Frase2, ... Frase11 ...)
        current_batch_start_index = i + 1
        formatted_input_string = format_batch_for_llm(batch_sentences, start_index=current_batch_start_index)
        
        print(f"\n--- A processar lote de frases: {current_batch_start_index} a {current_batch_start_index + len(batch_sentences) - 1} ---")

        try:
            print("A enviar para o LLM...")
            response_str = chain.invoke({"texto_entrada": formatted_input_string})
            # print(f"Resposta bruta do LLM: {response_str}") # Para depuração

            json_start = response_str.find('{')
            json_end = response_str.rfind('}') + 1
            if json_start != -1 and json_end != 0 and json_start < json_end:
                json_payload_str = response_str[json_start:json_end]
                llm_output_json = json.loads(json_payload_str)
                # print(f"JSON analisado: {llm_output_json}") # Para depuração

                # Processar cada frase do lote atual
                for k_batch_idx, original_sentence_words in enumerate(batch_sentences):
                    global_sentence_idx = i + k_batch_idx # Índice da frase no conjunto de dados completo
                    # Chave no JSON (ex: "Frase1", "Frase2", ... "Frase11")
                    sentence_key_in_json = f"Frase{global_sentence_idx + 1}" 
                    
                    true_tags_for_this_sentence = batch_true_tags[k_batch_idx]

                    if sentence_key_in_json in llm_output_json:
                        predicted_tags_for_this_sentence = llm_output_json[sentence_key_in_json]
                        
                        if isinstance(predicted_tags_for_this_sentence, list) and \
                           len(predicted_tags_for_this_sentence) == len(original_sentence_words):
                            # Tudo OK, adicionar às listas para o relatório
                            all_true_tags_for_report.extend(true_tags_for_this_sentence)
                            all_predicted_tags_for_report.extend(predicted_tags_for_this_sentence)
                            print(f"  + Predições para {sentence_key_in_json} (Frase original {global_sentence_idx + 1}) adicionadas ao relatório.")
                        else:
                            print(f"  ! Aviso: Discrepância ou formato inválido de etiquetas para {sentence_key_in_json} (Frase original {global_sentence_idx + 1}). Ignorando para o relatório.")
                            print(f"    Palavras ({len(original_sentence_words)}): {original_sentence_words}")
                            print(f"    Etiquetas verdadeiras: {true_tags_for_this_sentence}")
                            print(f"    Etiquetas previstas ({len(predicted_tags_for_this_sentence) if isinstance(predicted_tags_for_this_sentence, list) else 'N/A'}): {predicted_tags_for_this_sentence}")
                    else:
                        print(f"  ! Aviso: Nenhuma predição encontrada para {sentence_key_in_json} (Frase original {global_sentence_idx + 1}) na resposta do LLM. Ignorando para o relatório.")
            else:
                print(f"Erro: Não foi possível encontrar um objeto JSON válido na resposta do LLM para o lote. Resposta: {response_str}")

        except json.JSONDecodeError as json_e:
            print(f"Erro ao analisar a resposta JSON do LLM para o lote: {json_e}")
            print(f"String de resposta que causou o erro: '{response_str}'")
        except Exception as llm_e:
            print(f"Erro ao invocar a chain do LLM para o lote: {llm_e}")
            # Se um lote inteiro falhar, as etiquetas verdadeiras desse lote não serão adicionadas,
            # o que é o comportamento desejado para manter o alinhamento.
            continue
    
    print("\n\n--- Processamento de todos os lotes concluído ---")
    
    if all_true_tags_for_report and all_predicted_tags_for_report:
        if len(all_true_tags_for_report) == len(all_predicted_tags_for_report):
            print(f"Total de etiquetas para o relatório: {len(all_true_tags_for_report)}")
            
            # Definir as etiquetas que podem aparecer no relatório
            # (para garantir que todas as classes sejam consideradas, mesmo que algumas tenham 0 em precision/recall)
            defined_ner_labels = [
                "O", "B-Organizacao", "I-Organizacao", "B-Jurisprudencia", "I-Jurisprudencia",
                "B-Pessoa", "I-Pessoa", "B-Tempo", "I-Tempo", "B-Local", "I-Local",
                "B-Legislacao", "I-Legislacao"
            ]
            
            # Usar apenas as etiquetas que realmente apareceram nas predições ou nos dados verdadeiros
            # para evitar warnings no classification_report sobre labels sem amostras.
            # Ou pode-se usar 'defined_ner_labels' e aceitar os warnings/zeros.
            actual_labels_in_data = sorted(list(set(all_true_tags_for_report + all_predicted_tags_for_report)))
            
            print("\n--- Relatório de Classificação (sklearn.metrics.classification_report) ---")
            try:
                report = classification_report(
                    all_true_tags_for_report,
                    all_predicted_tags_for_report,
                    labels=actual_labels_in_data, # Ou defined_ner_labels se preferir
                    target_names=actual_labels_in_data, # Opcional, para nomes de classes
                    zero_division=0 # Define para 0 as métricas se houver divisão por zero (ex: sem predições para uma classe)
                )
                print(report)
            except ValueError as ve:
                print(f"Erro ao gerar relatório de classificação: {ve}")
                print("Isso pode acontecer se houver etiquetas nas predições que não estão nas etiquetas verdadeiras (ou vice-versa) e 'labels' não as cobrir.")
                print(f"Etiquetas verdadeiras únicas: {sorted(list(set(all_true_tags_for_report)))}")
                print(f"Etiquetas previstas únicas: {sorted(list(set(all_predicted_tags_for_report)))}")

        else:
            print("Erro Crítico: O número total de etiquetas verdadeiras e previstas não corresponde. Não é possível gerar o relatório.")
            print(f"Total de etiquetas verdadeiras: {len(all_true_tags_for_report)}")
            print(f"Total de etiquetas previstas: {len(all_predicted_tags_for_report)}")
    else:
        print("\nNenhuma etiqueta foi coletada ou prevista com sucesso para gerar o relatório de classificação.")


In [7]:
test_file_path = "test.txt"
process_ner_in_batches(test_file_path, 10)

A carregar frases e etiquetas verdadeiras do ficheiro: test.txt
Total de frases carregadas: 1389

--- A processar lote de frases: 1 a 10 ---
A enviar para o LLM...


KeyboardInterrupt: 